# Inferencia Gemma 3-4B con Function calling y multimodal

Function calling with Duckduckgo search and multimodal capabilities

## Setup

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install accelerate
!pip install duckduckgo-search

## Carga modelo

In [ ]:
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration

In [ ]:
model_id = "google/gemma-3-4b-it"

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto", token = hf_token
).eval()

processor = AutoProcessor.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
torch.cuda.empty_cache()

## Function Calling con Gemma

In [ ]:
from duckduckgo_search import DDGS

def search(query:str):
  """
  search results to the user query

  Args:
      query: user prompt to fetch search results
  """
  req = DDGS()
  response = req.text(query,max_results=4)
  context = ""
  for result in response:
    context += result['body']
  return context

In [ ]:
tools = [search]

In [ ]:
query = "which teams played Cricket ICC Champions Trophy 2025 finals and who won?" # this is a realtime query, this event occured on March 9th 2025

## Definir prompt para function calling

Function Calling follows these steps:

- Your application sends a prompt to the LLM along with function definitions
- The LLM analyzes the prompt and decides whether to respond directly or use defined functions
- If using functions, the LLM generates structured arguments for the function call
- Your application receives the function call details and executes the actual function
- The function results are sent back to the LLM
- The LLM provides a final response incorporating the function results

In [ ]:
conversation = [
    {
        "role": "system",
        "content": [{"type": "text", "text": """
        You are an expert search assistant. Use `search` to get the most accurate details.
        At each turn, if you decide to invoke any of the function(s), it should be wrapped with ```tool_code```. The python methods described below are imported and available,
        you can only use defined methods. The generated code should be readable and efficient. The response to a method will be wrapped in ```tool_output``` use it to call more tools or generate a helpful, friendly response.
        When using a ```tool_call``` think step by step why and how it should be used.

        The following Python methods are available:
        \`\`\`python
        def search(query:str):
          "
          search results to the user query

          Args:
             query: user prompt to fetch search results
          "
        \`\`\`

        User: \{user_message\}
        """}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": query}
        ]
    }
]

In [ ]:
inputs = processor.apply_chat_template(
            conversation,
            tools=tools,
            add_generation_prompt=True,
            return_dict=True,
            tokenize=True,
            return_tensors="pt",
).to(model.device, dtype=torch.bfloat16)

In [ ]:
outputs = model.generate(**inputs, max_new_tokens=512)

In [ ]:
output = processor.decode(outputs[0], skip_special_tokens=True)

In [ ]:
response = output.split("model")[-1]

In [ ]:
print(response)


```tool_code
search(query="Which teams played in the Cricket ICC Champions Trophy 2025 final and who won?")
```


## Ejecutar la función

In [ ]:
def extract_tool_call(text):
    import io
    from contextlib import redirect_stdout

    pattern = r"```tool_code\s*(.*?)\s*```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        code = match.group(1).strip()
        # Capture stdout in a string buffer
        f = io.StringIO()
        with redirect_stdout(f):
            result = eval(code)
        output = f.getvalue()
        r = result if output == '' else output
        return f'```tool_output\n{r}\n```'''
    return None

In [ ]:
keyword = extract_tool_call(response)

In [ ]:
keyword

"```tool_output\nThe ninth edition of the Champions Trophy 2025 saw India being crowned as the winners on 9th March 2025 after they overcame New Zealand in the final. Several exceptional performers lit up the tournament with the bat and ball. The best of them made it to the Team of the Tournament. Here's what the side looks like: 1. Rachin Ravindra (New Zealand)The 2025 ICC Champions Trophy was the ninth edition of the ICC Champions Trophy, a quadrennial ODI cricket tournament organised by the International Cricket Council (ICC). In November 2021 as part of the 2024-2031 ICC men's hosts cycle, the ICC announced that the 2025 Champions Trophy would be played in Pakistan. [4]On 19 December 2024, following an agreement between Board of Control for ...Official ICC Champions Trophy, 2025 Cricket website - live matches, scores, news, highlights, commentary, rankings, videos and fixtures from the International Cricket Council. ... 2025. ICC announces Champions Trophy 2025 Team of the Tourname

In [ ]:
prompt = [{
        "role": "user",
        "content": [
            {"type": "text", "text": f"Results from the search:{keyword} User_query : {query}"}
        ]
    }
]

In [ ]:
final_prompt = processor.apply_chat_template(
            prompt,
            tools=tools,
            add_generation_prompt=True,
            return_dict=True,
            tokenize=True,
            return_tensors="pt",
).to(model.device, dtype=torch.bfloat16)

In [ ]:
response = model.generate(**final_prompt, max_new_tokens=512)

In [ ]:
print(processor.decode(response[0], skip_special_tokens=True))

user
Results from the search:```tool_output
The ninth edition of the Champions Trophy 2025 saw India being crowned as the winners on 9th March 2025 after they overcame New Zealand in the final. Several exceptional performers lit up the tournament with the bat and ball. The best of them made it to the Team of the Tournament. Here's what the side looks like: 1. Rachin Ravindra (New Zealand)The 2025 ICC Champions Trophy was the ninth edition of the ICC Champions Trophy, a quadrennial ODI cricket tournament organised by the International Cricket Council (ICC). In November 2021 as part of the 2024-2031 ICC men's hosts cycle, the ICC announced that the 2025 Champions Trophy would be played in Pakistan. [4]On 19 December 2024, following an agreement between Board of Control for ...Official ICC Champions Trophy, 2025 Cricket website - live matches, scores, news, highlights, commentary, rankings, videos and fixtures from the International Cricket Council. ... 2025. ICC announces Champions Troph

## Multimedia. Helper functions

In [ ]:
from PIL import Image
import cv2
from IPython.display import Markdown, HTML


def resize_image(image_path):
    img = Image.open(image_path)

    target_width, target_height = 640, 640
    # Calculate the target size (maximum width and height).
    if target_width and target_height:
        max_size = (target_width, target_height)
    elif target_width:
        max_size = (target_width, img.height)
    elif target_height:
        max_size = (img.width, target_height)

    img.thumbnail(max_size)

    return img


def get_model_response(img: Image, prompt: str, model, processor):
    # Prepare the messages for the model.
    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a helpful assistant. Reply only with the answer to the question asked, and avoid using additional text in your response like 'here's the answer'."}]
        },
        {
            "role": "user",
            "content": [
                {"type": "image", "image": img},
                {"type": "text", "text": prompt}
            ]
        }
    ]

    # Tokenize inputs and prepare for the model.
    inputs = processor.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    ).to(model.device, dtype=torch.bfloat16)

    input_len = inputs["input_ids"].shape[-1]

    # Generate response from the model.
    with torch.inference_mode():
        generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
        generation = generation[0][input_len:]

    # Decode the response.
    response = processor.decode(generation, skip_special_tokens=True)
    return response


def extract_frames(video_path, num_frames):
    """
    The function is adapted from:
    https://github.com/merveenoyan/smol-vision/blob/main/Gemma_3_for_Video_Understanding.ipynb
    """
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open video file.")
        return []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Calculate the step size to evenly distribute frames across the video.
    step = total_frames // num_frames
    frames = []

    for i in range(num_frames):
        frame_idx = i * step
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if not ret:
            break
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        timestamp = round(frame_idx / fps, 2)
        frames.append((img, timestamp))

    cap.release()
    return frames


def show_video(video_path, video_width = 600):
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  video_html = f""""""
  return HTML(video_html)

## Inference on images

In [ ]:
!wget https://raw.githubusercontent.com/NSTiwari/Inference-Gemma-3/main/assets/image_1.png -O /content/image_1.png
!wget https://raw.githubusercontent.com/NSTiwari/Inference-Gemma-3/main/assets/image_2.png -O /content/image_2.png
!wget https://raw.githubusercontent.com/NSTiwari/Inference-Gemma-3/main/assets/image_3.png -O /content/image_3.png
!wget https://raw.githubusercontent.com/NSTiwari/Inference-Gemma-3/main/assets/image_4.png -O /content/image_4.png

In [ ]:
image_file = 'image_1.png' 
prompt = "Describe the image." 

img = resize_image(image_file)
display(img)
response = get_model_response(img, prompt, model, processor)
display(Markdown(response))

In [ ]:
image_file = 'image_2.png' 
prompt = "Identify the famous landmark and location." 

img = resize_image(image_file)
display(img)
response = get_model_response(img, prompt, model, processor)
display(Markdown(response))

In [ ]:
image_file = 'image_3.png' 
prompt = "que es este dibujo ? " 

img = resize_image(image_file)
display(img)
response = get_model_response(img, prompt, model, processor)
display(Markdown(response))

In [ ]:
# Mathematical reasoning

from PIL import Image
from IPython.display import Markdown

image_file = 'image_4.png' 
prompt = "What is the value of x?" 

img = resize_image(image_file)
display(img)
response = get_model_response(img, prompt, model, processor)
display(Markdown(response))